In [1]:
# analysis notebook 

# see opt_fun.py for how these are calculated 
# see the individual files for the data 

In [2]:
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from IPython.display import display
import numpy as np

In [3]:
from analysis_fun import *

In [4]:
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

username = "jnwagwu"
api_key = "119wNvkoTSGwHIxORQ85"

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Import Data

In [5]:
def get_sub_dirs(path):
    sub_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return sub_dirs

root_path = os.getcwd()
opt_models_path= os.path.join(root_path,"opt_models")

In [6]:
get_sub_dirs(opt_models_path)

['light_0',
 'floor_2',
 'roof_2',
 'test_01',
 'shgc_b1',
 'window_u',
 'wall_b1',
 'floor_b1',
 'daylight_0',
 'roof_b1',
 'equip_0',
 'clgsetp_0',
 'roof_0',
 'htgsetp_0']

In [7]:
group_models = {
    "SHGC_and_True_Exteriors": ["shgc_b1", "roof_b1", "wall_b1", "floor_b1"],
    "Exteriors": ["wall_b1", "floor_2", "roof_0"],
    "Exteriors and Windows": ["wall_b1", "floor_2", "roof_0", "window_u", "shgc_b1"],
    "Windows": ["window_u", "shgc_b1"],
    "Electric Interior Gains": ["light_0", "equip_0"],
    "Setpoints": ["clgsetp_0", "htgsetp_0"],
    "Daylight": ["daylight_0"]

}

group_models["All"] = group_models["Exteriors and Windows"] + group_models["Electric Interior Gains"] + group_models["Setpoints"] + group_models["Daylight"]
print(len(group_models["All"]))


10


In [8]:
# labels 
labels = {
    "Wall R [ft2-ºF-hr/Btu]" : "wall",
    "Floor R [ft2-ºF-hr/Btu]": "floor",
    "Roof R [ft2-ºF-hr/Btu]": "roof",
    "Window SHGC": "shgc",
    "Window U-Factor [Btu/ft2-F-hr]": "window",
    "Light Power Density [W/ft2]": "light",
    "Equip. Power Density [W/ft2]": "equip",
    "Heating Setp. [ºF]": "htgsetp",
    "Cooling Setp. [ºF]": "clgsetp",
    "Daylight Controls": "daylight"

}

labels_no_units = {
    "Wall R" : "wall",
    "Floor R": "floor",
    "Roof R": "roof",
    "Window SHGC": "shgc",
    "Window U-Factor": "window",
    "Light Power Density": "light",
    "Equip. Power Density ": "equip",
    "Heating Setp.": "htgsetp",
    "Cooling Setp.": "clgsetp",
    "Daylight Controls": "daylight"

}

labels_rev = {v: k for k,v in labels.items()}
labels_nounits_rev = {v: k for k,v in labels_no_units.items()}
# labels_rev

In [9]:
# assign units 
groups  = {
    "r_val": ["wall", "floor", "roof"],
    "u_val": ["window"],
    "efficiency": ["light", "equip"],
    "setp": ["setp"],
    "none": ["shgc", "daylight"]
}

units = {
    "u_val": "W/m2-K",
    "r_val": "m2-K/W",
    "efficiency": "W/m2",
    "setp": "C",
    "none": ""
}

# https://betterhomesbc.ca/products/what-is-r-or-rsi-value/#:~:text=The%20R%2DValue%20is%20an,transfer%20through%20the%20insulating%20material.

r_val_si_to_imperial_factor = 5.678 # multiply the si r value by this number

def u_val_si_to_imperial(u_val):
    return u_val * (1/ r_val_si_to_imperial_factor )

def r_val_si_to_imperial(r_val):
    return r_val * r_val_si_to_imperial_factor 

def celcius_to_f(temp):
    return temp * (9/5) + 32

def w_m2_to_w_ft2(energy):
    return energy * (1/10.76)

units_imperial = {
    "u_val": ("Btu/ft2-F-hr", u_val_si_to_imperial ),
    "r_val": ("ft2-F-hr/Btu", r_val_si_to_imperial ),
    "efficiency": ("W/ft2", w_m2_to_w_ft2),
    "setp": ("F", celcius_to_f),
    "none": ("", lambda a: a),
}
keys = list(group_models["All"])
key_units = {}
for key in keys:
    # assign key to group 
    for k, v in groups.items():
        for string in v:
            if string in key:
                key_units[key] = (units[k], units_imperial[k])
# key_units

In [11]:
def get_group_csv(group_model):
    units_data = key_units[group_model]
    conversion_function = units_data[1][1]
    # print(units)
    group_model_path = os.path.join(opt_models_path, group_model )
    csv_path = os.path.join(group_model_path, "data.csv")
    df = pd.read_csv(csv_path, index_col="Unnamed: 0")

    # add imperial values 
    df["imperial_vals"] = conversion_function(df["vals"])
    df["imperial_vals_units"] = units_data[1][0]
    
    return df

In [12]:
def get_group(group_of_interest, example=False):
    group_data = {model: get_group_csv(model) for model in group_models[group_of_interest]}
    
    group_keys = list(group_data.keys())
    # print(group_keys)
    if example:
        first_data = list(group_data.keys())[0]
        display(group_data[first_data])

    
    # simple group keys
    simple_gk = [k.split("_")[0] for k in group_keys]

    # get labels 
    group_labels = {real_key: labels_rev[group_key] for group_key, real_key in zip(simple_gk, group_keys) }
    group_labels_no_units = {real_key: labels_nounits_rev[group_key] for group_key, real_key in zip(simple_gk, group_keys) }
    

        
    return group_keys, group_data, group_of_interest, group_labels, group_labels_no_units

    # envelope 

In [42]:
df = get_group_csv("light_0")
df

,elect,hot,chill,elect_c,hot_c,chill_c,total,total_c,vals,perc_change,imperial_vals,imperial_vals_units
-98.0,109233.580993,80590.329074,58812.485168,18935.52,10999.42,7023.68,248636.395235,36958.62,0.264792,-98.0,0.024609,W/ft2
-75.0,116969.246724,77959.012279,59926.025840,20413.87,10640.29,7156.66,254854.284843,38210.82,3.309902,-75.0,0.307612,W/ft2
-50.0,125420.944070,75351.870120,61293.319251,22031.85,10284.45,7319.95,262066.133441,39636.25,6.619805,-50.0,0.615224,W/ft2
-25.0,133905.128470,73366.896826,63111.157208,23678.89,10013.53,7537.05,270383.182504,41229.47,9.929707,-25.0,0.922835,W/ft2
-10.0,139028.051363,72322.248594,64383.078779,24667.23,9870.95,7688.95,275733.378736,42227.13,11.915649,-10.0,1.107402,W/ft2
-5.0,140739.082088,71989.089240,64821.405640,24997.92,9825.48,7741.29,277549.576968,42564.69,12.577629,-5.0,1.168925,W/ft2
-2.5,141595.866928,71824.769536,65045.047025,25163.59,9803.05,7768.00,278465.683490,42734.64,12.908620,-2.5,1.199686,W/ft2
-1.0,142110.160798,71727.320912,65179.554247,25262.65,9789.75,7784.07,279017.035958,42836.47,13.107214,-1.0,1.218143,W/ft2
0.0,142453.408250,71662.765076,65269.397110,25328.67,9780.94,7794.79,279385.570436,42904.40,13.239610,0.0,1.230447,W/ft2


In [49]:
x = np.array(df["imperial_vals"])
y = np.array(df["total"])
y

array([248636.39523507, 254854.28484349, 262066.13344083, 270383.18250382,
       275733.3787363 , 277549.5769678 , 278465.68348967, 279017.03595783,
       279385.57043564])

In [50]:
A = np.vstack([x, np.ones(len(x))]).T
m, c = np.linalg.lstsq(A, y, rcond=None)[0]
print(m, c)

25894.091137955704 247138.87354276015


In [55]:
feasible_opt = m*0.44 + c
baseline = df.loc[0]["total"]
savings = feasible_opt - baseline
savings

-20853.296792177658

In [56]:
def calc_feasible_savings(df, feasible_val):
    # energy 
    x = np.array(df["imperial_vals"])
    y = np.array(df["total"])
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    feasible_opt = m*0.44 + c
    baseline = df.loc[0]["total"]
    e_savings = feasible_opt - baseline

    # cost 
    x = np.array(df["imperial_vals"])
    y = np.array(df["total_c"])
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    feasible_opt = m*0.44 + c
    baseline = df.loc[0]["total_c"]
    cost_savings = feasible_opt - baseline

    return (e_savings, cost_savings)
    
    

# Exterior Surfaces

In [13]:
def saveImagePlotly(fig, fig_name):
    fig_name_png = fig_name + ".png"
    # fig_path = os.path.join(root, "figs_final_pres_03_07", fig_name_png )
    final_paper_path = "/Users/julietnwagwuume-ezeoke/My Drive/CEE256_BuildSys/final_256/Deliverables/PaperFigs"
    fig_path = os.path.join(final_paper_path, fig_name_png)
    fig.write_image(fig_path, width=1200, format="png", engine="kaleido",
                    scale=2)

In [14]:
es_data = get_group("Exteriors", False)

In [15]:
fig = four_compare(es_data[0], es_data[1], es_data[2], es_data[3])
saveImagePlotly(fig, "test")

Link  https://plotly.com/~jnwagwu/161/


In [16]:
correlations(es_data[1], es_data[2])

# SHGC + True Exteriors

In [17]:
# shgc = get_group("SHGC_and_True_Exteriors", False)
# data = shgc
# four_compare(data[0], data[1], data[2], data[3])
# correlations(data[1], data[2])


# Ext Surfaces + SHGC + U Window

In [18]:
data = get_group("Exteriors and Windows", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/163/


# Lighting, Equip

In [19]:
data = get_group("Electric Interior Gains", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/165/


# Setpoints

In [20]:
data = get_group("Setpoints", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/104/


# Daylight Controls

In [21]:
data = get_group("Daylight", False)
four_compare(data[0], data[1], data[2], data[3])
# correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/108/


# All

In [22]:
data = get_group("All", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/112/


In [23]:
# todo bar graph with best case energy and equip savings for each 
# show feasible on this graph 
# graph with best case after feasibilty analysis 
# combinations 

In [24]:
group_data = data[1]
group_data["clgsetp_0"]

,elect,hot,chill,elect_c,hot_c,chill_c,total,total_c,vals,perc_change,imperial_vals,imperial_vals_units
0.0,142453.408250,71662.765076,65269.397110,25328.67,9780.94,7794.79,279385.570436,42904.40,29.814815,0.0,85.666667,F
1.0,142294.173883,71564.224684,64225.513191,25275.21,9767.49,7670.13,278083.911757,42712.83,30.112963,1.0,86.203333,F
2.5,142104.495645,71428.555387,62787.338560,25203.02,9748.97,7498.38,276320.389593,42450.37,30.560185,2.5,87.008333,F
5.0,141899.559867,71234.462712,60784.825053,25123.38,9722.48,7259.23,273918.847632,42105.09,31.305556,5.0,88.350000,F
10.0,141737.930000,70957.400071,57896.626712,25035.55,9684.67,6914.30,270591.956783,41634.52,32.796296,10.0,91.033333,F
25.0,141696.002425,70845.621610,55974.825077,24994.49,9669.41,6684.79,268516.449112,41348.69,37.268519,25.0,99.083333,F
50.0,141696.002379,70845.621609,55974.816964,24994.49,9669.41,6684.79,268516.440953,41348.69,44.722222,50.0,112.500000,F
98.0,141696.002379,70845.621609,55974.816964,24994.49,9669.41,6684.79,268516.440953,41348.69,59.033333,98.0,138.260000,F
75.0,141696.002379,70845.621609,55974.816964,24994.49,9669.41,6684.79,268516.440953,41348.69,52.175926,75.0,125.916667,F


In [25]:

max_vals = {}
for ix, k in enumerate(group_data.keys()):
    x_ = group_data[k]["total_c"] - group_data[k].loc[0]["total_c"]
    z_ = group_data[k]["total"] - group_data[k].loc[0]["total"]
    y_ = list(group_data[k]["imperial_vals"])
    
    # concentrate on possibility of savings
    x = [-1*min(val, 0) for val in x_]
    z = [-1*min(val, 0) for val in z_]

    max_save = max(x)
    max_save_ix = x.index(max_save)
    max_save_val = y_[max_save_ix]
    max_save_energy = z[max_save_ix]
    max_vals[k] = (max_save, max_save_val, max_save_energy)
print(max_vals)

{'wall_b1': (331.45000000000437, 39.58605633802817, 2480.1814389269857), 'floor_2': (698.0999999999985, 0.39985915492957913, 5318.357948485063), 'roof_0': (496.77999999999156, 39.58605633802817, 3647.0607679850655), 'window_u': (1101.479999999996, 0.004226840436773511, 8241.055381817569), 'shgc_b1': (634.1800000000003, 0.0061999999999999, 4893.969440043496), 'light_0': (5945.779999999999, 0.024608940172031572, 30749.175200569152), 'equip_0': (11057.04, 0.03717472118959095, 55328.06305150967), 'clgsetp_0': (1555.7099999999991, 99.08333333333346, 10869.121323386906), 'htgsetp_0': (5311.669999999998, 32.692, 38874.54386635299), 'daylight_0': (1058.2099999999991, 1, 5700.413144911989)}


## Line Up

In [26]:
keys = list(data[3].values())
keys

['Wall R [ft2-ºF-hr/Btu]',
 'Floor R [ft2-ºF-hr/Btu]',
 'Roof R [ft2-ºF-hr/Btu]',
 'Window U-Factor [Btu/ft2-F-hr]',
 'Window SHGC',
 'Light Power Density [W/ft2]',
 'Equip. Power Density [W/ft2]',
 'Cooling Setp. [ºF]',
 'Heating Setp. [ºF]',
 'Daylight Controls']

In [27]:
def saveImagePlotly(fig, fig_name):
    fig_name_png = fig_name + ".png"
    # fig_path = os.path.join(root, "figs_final_pres_03_07", fig_name_png )
    final_paper_path = "/Users/julietnwagwuume-ezeoke/My Drive/CEE256_BuildSys/final_256/Deliverables/PaperFigs"
    fig_path = os.path.join(final_paper_path, fig_name_png ) # width=1200, height=350, 
    fig.write_image(fig_path, format="png", engine="kaleido",  scale=2)

In [28]:
fig = make_subplots(
        rows=1, cols=1,
        subplot_titles=(
            "Maximum Observed Cost Savings for Each Variable",
            "Feasible Cost Savings [$]",
            # "Energy Savings [$]",
            # "Feasible Energy "
        ))

x = list(data[4].values())
y = [val[0] for val in max_vals.values()]
text = [f"{np.round(v[1],2)} {key_units[k][1][0]}" for k,v in max_vals.items()]
fig.add_trace(go.Bar(x=x, y=y, text=text, textposition="auto"), row=1, col=1)
fig.update_layout(yaxis_title = "Annual Cost Savings [$]")
fig.show()

saveImagePlotly(fig, f"Cost_Save_Comparisons")

In [29]:
fig = make_subplots(
        rows=1, cols=1,
        subplot_titles=(
            # "Cost Savings [$]",
            # "Feasible Cost",
            "Maximum Observed Energy Savings for Each Variable",
             "Maximum Possible Energy Savings for Each Variable"
        ))


# y = [val[0] for val in max_vals.values()]
y2 = [val[2] for val in max_vals.values()]
# fig.add_trace(go.Bar(x=x, y=y, text=text, textposition="auto"), row=1, col=1)
fig.add_trace(go.Bar(x=x, y=y2, text=text, textposition="auto"), row=1, col=1)


fig.update_layout(yaxis_title = "Annual Energy Savings [kWh]")
fig.show()

saveImagePlotly(fig, f"Energy_Save_Comparisons")

# Feasibility Analysis 

In [ ]:
def calc_feasible_savings(df, feasible_val):
    # energy 
    x = np.array(df["imperial_vals"])
    y = np.array(df["total"])
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    feasible_opt = m*0.44 + c
    baseline = df.loc[0]["total"]
    e_savings = feasible_opt - baseline

    # cost 
    x = np.array(df["imperial_vals"])
    y = np.array(df["total_c"])
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    feasible_opt = m*0.44 + c
    baseline = df.loc[0]["total_c"]
    cost_savings = feasible_opt - baseline

    return (e_savings, cost_savings)

In [58]:
group_data.keys()

dict_keys(['wall_b1', 'floor_2', 'roof_0', 'window_u', 'shgc_b1', 'light_0', 'equip_0', 'clgsetp_0', 'htgsetp_0', 'daylight_0'])

In [57]:
for k, v in zip(group_data.items(), feasible:

{'wall_b1':              elect           hot          chill   elect_c    hot_c   chill_c  \
 -98  110303.585613  49711.300809  104149.821091  20396.60  6784.88  12438.09   
 -50  107453.878982  22565.561191   80634.937480  19910.75  3079.88   9629.82   
 -25  107250.927219  20865.901685   79546.103348  19872.63  2847.90   9499.79   
  0   107143.924137  19959.899623   78974.377892  19852.73  2724.24   9431.51   
  25  107077.444179  19398.951814   78620.299120  19839.96  2647.68   9389.23   
  50  107031.913910  19018.005027   78377.334576  19832.02  2595.69   9360.21   
  98  106975.964947  18548.616444   78073.438823  19821.49  2531.62   9323.92   
 
              total   total_c      vals  perc_change  imperial_vals  \
 -98  264164.707514  39619.57  0.070423          -98       0.399859   
 -50  210654.377653  32620.45  1.760563          -50       9.996479   
 -25  207662.932252  32220.32  2.640845          -25      14.994718   
  0   206078.201652  32008.48  3.521127            0   

In [30]:
# feasability analysis 

# Wall R-Value [units]
# Roof R-Value [units] 
# Floor R-Value [units]
# Solar Heat Gain Coefficient (SHGC)
# U-Factor [units]
# Equipment Watts per Zone Floor Area [units]
# Lighting Watts per Zone Floor Area [units]
# Cooling Setpoint [units]
# Heating Setpoint [units]
# Daylight Controls Applied



In [31]:
# equip power densitt 